In [2]:
print("ok")

ok


In [3]:
%pwd


'c:\\Users\\HP\\Documents\\RAG\\Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS-\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\HP\\Documents\\RAG\\Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS-'

In [6]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\HP\miniconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    
    return documents

In [8]:
extracted_data = load_pdf_files("data")

In [9]:
len(extracted_data)

615

In [10]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: list[Document] = []
    for doc in docs:
        src=doc.metadata.get("source")
        minimal_docs.append(Document(page_content=doc.page_content, metadata={"source": src}))
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
print(minimal_docs[0])

page_content='The GALE
ENCYCLOPEDIA of
AlTERNA TIVE
MEDICINE
SECOND EDITION' metadata={'source': 'data\\altmedbook1.pdf'}


In [13]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
       
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [14]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 5982


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embedding = download_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_17696\1812119473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
vector = embedding.embed_query("Hello world")
print(vector)

[-0.034477293491363525, 0.031023185700178146, 0.006734919268637896, 0.026108955964446068, -0.03936200588941574, -0.16030244529247284, 0.06692398339509964, -0.00644145580008626, -0.047450482845306396, 0.014758873730897903, 0.07087531685829163, 0.05552761256694794, 0.019193336367607117, -0.026251327246427536, -0.010109526105225086, -0.026940451934933662, 0.02230745740234852, -0.02222668007016182, -0.14969263970851898, -0.017492998391389847, 0.007676251698285341, 0.05435226485133171, 0.003254401497542858, 0.031725890934467316, -0.08462139964103699, -0.029405971989035606, 0.051595598459243774, 0.04812406003475189, -0.003314854810014367, -0.05827920511364937, 0.04196922481060028, 0.022210687398910522, 0.1281888335943222, -0.02233893983066082, -0.011656275019049644, 0.06292839348316193, -0.032876357436180115, -0.0912260189652443, -0.03117534890770912, 0.05269956961274147, 0.04703487083315849, -0.08420306444168091, -0.030056191608309746, -0.02074483036994934, 0.009517822414636612, -0.00372179

In [17]:
print(len(vector))

384


In [18]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [19]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [20]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)


In [21]:
from pinecone import ServerlessSpec
index_name = "medical"  
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)

In [22]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(documents=texts_chunk, embedding=embedding, index_name=index_name)

In [23]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(embedding=embedding, index_name=index_name)

In [24]:
dswith= Document(
    page_content="Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.",
    metadata={"source": "data/Hypertension.pdf"}
)

In [25]:
docsearch.add_documents(documents=[dswith])

['3a0cef54-2c6e-4683-8641-138d20a54d96']

In [26]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [27]:
retrieved_docs=retriever.invoke("What is hypertension?")
retrieved_docs

[Document(id='5c91a8d5-ed44-4812-b626-1b93315d1e4b', metadata={'source': 'data/Hypertension.pdf'}, page_content='Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.'),
 Document(id='d4f3a9b3-d550-4d0b-ac0a-1954fd2e54c5', metadata={'source': 'data/Hypertension.pdf'}, page_content='Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.'),
 Document(id='7d9e207f-8466-42a3-9b05-5d9d58ac0761', metadata={'source': 'data/Hypertension.pdf'}, page_content='Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.')]

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Initialisation du modèle via OpenRouter
llm = ChatOpenAI(
    model_name="gpt-4",              # modèle disponible via OpenRouter
    temperature=0,
    max_tokens=200,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="OPENAI_API_KEY"  # <-- ici ta vraie clé
)

# Envoyer un message


C:\Users\HP\AppData\Local\Temp\ipykernel_17696\2359792082.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [30]:
system_prompt=(
    "you are a helpful medical assistant. Use the following context to answer the user's question accurately and concisely. If the information is not available in the context, respond with 'I don't know.'"
    "use."
    "\n\n"
    "{context}"

)
prompt= ChatPromptTemplate.from_messages([("system", system_prompt), ("human","{input}"),])

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
print(type(rag_chain))


<class 'langchain_core.runnables.base.RunnableBinding'>


In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import HumanMessage

# 1️⃣ Créer ton LLM via OpenRouter
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0,
    max_tokens=200,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key="OPENAI_API_KEY"
)

# 2️⃣ Ton retriever (index vectoriel)


# 3️⃣ Créer la RAG chain correctement avec LLM fourni
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,          # <-- ici on passe notre LLM OpenRouter
    retriever=retriever
)

# 4️⃣ Appel
response = rag_chain.run("What is hypertension?")
print(response)


C:\Users\HP\AppData\Local\Temp\ipykernel_17696\1568391218.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run("What is hypertension?")


AuthenticationError: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}

In [34]:
response = rag_chain.run("What is Acne")
print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'No cookie auth credentials found', 'code': 401}}

In [ ]:
query = "What is hypertension?"
docs = docsearch.similarity_search(query, k=2)
print("Docs récupérés :", docs)

if docs:
    response = rag_chain.run(query)
    print("Réponse :", response)
else:
    print("Aucun document trouvé !")


Docs récupérés : [Document(id='d4f3a9b3-d550-4d0b-ac0a-1954fd2e54c5', metadata={'source': 'data/Hypertension.pdf'}, page_content='Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.'), Document(id='5c91a8d5-ed44-4812-b626-1b93315d1e4b', metadata={'source': 'data/Hypertension.pdf'}, page_content='Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.')]
Réponse : Hypertension is a chronic medical condition in which the blood pressure in the arteries is persistently elevated. It is a major risk factor for heart disease, stroke, and other cardiovascular problems.
